In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=0)

In [4]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [5]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [24]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
])
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs=1000, verbose = 0)

In [25]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 120ms/step - loss: 0.6061 - accuracy: 0.8333


[0.6060981154441833, 0.8333333134651184]

In [8]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [9]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 89ms/step


array([[0.5522376 ],
       [0.5380268 ],
       [0.55628324],
       [0.5743889 ],
       [0.57038087],
       [0.5135339 ]], dtype=float32)

In [10]:
y_test

2     1
22    1
14    1
17    1
5     1
11    0
Name: bought_insurance, dtype: int64

In [12]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.8187736],
        [0.515923 ]], dtype=float32),
 array([-0.6910309], dtype=float32))

In [15]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

In [16]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0] * age + coef[1] * affordibility + intercept
  return sigmoid(weighted_sum)

In [17]:
prediction_function(0.47 ,1)

0.5522376026014306

In [18]:
prediction_function(0.40 ,1)

0.53802681844248

In [19]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    
    return -np.mean(y_true * np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))

In [20]:
def sigmoid_numpy(x):
  return 1 / (1 + np.exp(-x))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [21]:
class myNN:
    def __init__(self): # Here is my constructor
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold): # Here is my fit method in function, self is always firs argument
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_thresold) # It will return me a tuple
        print(f"Final weights and bias : w1: {self.w1}, w2: {self.w2}, bias: {self.bias}") # So my gradient_descent will return my weights an bias 
        
    def predict(self, X_test): # Its my prediction function
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias # Its my weighted_sum from pattern  
        return sigmoid_numpy(weighted_sum) # And its activated by sigmoid_numpy

    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold): # This function help finding weights
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
       
        n = len(age)
        
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1 / n) * np.dot(np.transpose(age), (y_predicted - y_true)) 
            w2d = (1 / n) * np.dot(np.transpose(affordability), (y_predicted - y_true)) 
            bias_d = np.mean(y_predicted - y_true)
            
            w1 = w1 - learning_rate * w1d
            w2 = w2 - learning_rate * w2d
            bias = bias - learning_rate * bias_d
            
            if i%50 == 0: # The lines will be printing every fiftieth epoch
                print(f'Epoch : {i}, w1 : {w1}, w2 : {w2}, bias : {bias}, loss : {loss}')
            
            if loss <= loss_thresold:
                print(f'Epoch : {i}, w1 : {w1}, w2 : {w2}, bias : {bias}, loss : {loss}')
                break

        return w1, w2, bias
  

In [23]:
customModel = myNN() # I create object of my class 
customModel.fit(X_train_scaled, y_train, epochs = 500, loss_thresold = 0.6163) # Here is my fit method

Epoch : 0, w1 : 0.9644915668168719, w2 : 0.9224250019701857, bias : -0.1521744437835525, loss : 0.7960362503355486
Epoch : 8, w1 : 0.8978991061148435, w2 : 0.6612102292494013, bias : -0.796166156177148, loss : 0.6115548071590621
Final weights and bias : w1: 0.8978991061148435, w2: 0.6612102292494013, bias: -0.796166156177148


In [26]:
coef, intercept # It is comaparation with my keras model

(array([[0.8187736],
        [0.515923 ]], dtype=float32),
 array([-0.6910309], dtype=float32))

In [27]:
customModel.predict(X_test_scaled) # And here is prediction comparation with my keras model

2     0.571275
22    0.555817
14    0.575668
17    0.595277
5     0.590944
11    0.529081
dtype: float64

In [28]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 54ms/step


array([[0.5522376 ],
       [0.5380268 ],
       [0.55628324],
       [0.5743889 ],
       [0.57038087],
       [0.5135339 ]], dtype=float32)